In [5]:
## you can run pip3 install -r requirements.txt to install all the packages
## but you need to install tensorflow or pytorch or keras manually

import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import threading, os, sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # try to use CPU only

# addin path to import IQ module
sys.path.append('../../')
import src.IQ as IQ

In [6]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/admin")
BLE = myclient["BLE"]

def query(collection, filter:dict, addFrameColumn=True):
    df =  pd.DataFrame(list(collection.find(filter)))
    if addFrameColumn:
        df['frame'] = df.apply(lambda x: x['I'] + np.dot(x['Q'],1j), axis=1)
    return df.copy()

In [ ]:
from sklearn.model_selection import train_test_split

filtering = {''}
df = query(BLE['onBody'], {'pos':'static','antenna_side':'left'})
min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:2000])
print(len(df['frame'][0]))



In [ ]:
iq = IQ.IQ(Fc=2439810000+.1e4)

def configCreator(downSampleRate = 1, cutoff = 2e6):
    downSampleRate= max(downSampleRate, 1)
    return {                                      
            iq.butter:{'Fs': iq.Fs/downSampleRate, "cutoff": cutoff},
            iq.downSample:{'downSampleRate':downSampleRate, "shift": 0},
            iq.demodulate:{'Fs': iq.Fs},
           }

methods = configCreator(downSampleRate=  1)
df['data'] = iq.apply(methods = methods, frame = df)

############### Defing and normalizing the input #############
# df['normalized_input_feature'] = df['data'].apply(lambda x: fft_normalized(x, threshold = 1))
df['normalized_input_feature'] = df['data'].apply(lambda x: x[0:2000])
df['normalized_input_feature_realImage'] = df['normalized_input_feature'].apply(lambda x: normalized(np.concatenate([np.real(x[0:2000]), np.imag(x[0:2000])])))
##################################################################################

X_train, X_test, y_train, y_test = train_test_split(df['normalized_input_feature'], df['dvc'], test_size=0.2, random_state=42)